In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [1]:
###
### я считал все на кластере рабочем, 
### так как учебный кластер был недоступен. 
### 

In [93]:
# film for calculation: 302 
# L.A. Confidential (1997) 

In [50]:
src = pd.read_csv('u.data', sep='\t', header=None).rename(columns={0:'userid', 1:'itemid', 2:'rate', 3:'timestampcolumn'})
src.sample()

,userid,itemid,rate,timestampcolumn
95497,894,628,3,880416102


In [60]:
df = spark.createDataFrame(src)[['itemid', 'rate']]
df

DataFrame[itemid: bigint, rate: bigint]

In [61]:
df = df.repartition(10)
df.rdd.getNumPartitions() 

10

In [81]:
hist_all = df[['rate']].groupBy('rate').count().toPandas().sort_values( by=['rate'])['count'].tolist()
hist_film = df[df['itemid']=='302'][['rate']].groupBy('rate').count().toPandas().sort_values( by=['rate'])['count'].tolist()

In [82]:
hist_all

[6110, 11370, 27145, 34174, 21201]

In [83]:
hist_film

[2, 10, 46, 119, 120]

In [87]:
result = {'hist_all': hist_all, 'hist_film':hist_film}

In [92]:
import json 
result = {'hist_all': hist_all, 'hist_film':hist_film}
with open('result.json', 'w') as f: 
    json.dump(result, f) 

In [66]:
df[df['itemid']=='302'][['rate']].groupBy('rate').count()

[Row(rate=5, count=120),
 Row(rate=1, count=2),
 Row(rate=3, count=46),
 Row(rate=2, count=10),
 Row(rate=4, count=119)]